# Dogs vs. Cats Redux: Kernels Edition

## 2. 모델구성 및 학습

In [1]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import time
import re

# 구현모듈
import data
import model
import train

### 2.1 학습 데이터 생성 함수

In [2]:
def getFiles(dir, word):
    files =   [dir+i for i in os.listdir(dir) if word in i]
    files.sort(key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
    return files

In [3]:
def getDogCatData(image_dir, test_size=0.3):
    dog_files =  getFiles(image_dir, 'dog')
    cat_files =  getFiles(image_dir, 'cat')
    print("dog_files: ", len(dog_files))
    print("cat_files: ", len(cat_files))
    
    dog_files = dog_files[:200]
    cat_files = cat_files[:200]

    dog_cat_files = dog_files + cat_files

    label_dog = [0 for i in range(len(dog_files))]
    label_cat = [1 for i in range(len(cat_files))]
    label_one_hot = np.eye(2)[label_dog + label_cat]

    return train_test_split (dog_cat_files, label_one_hot, test_size=test_size, random_state=42)

### 2.2 학습데이터와 valid 데이터 분리

In [4]:
DATA_DIR = "../data/"
TRAIN_DIR = DATA_DIR + "train_resize/"

x_train, x_valid, y_train, y_valid = getDogCatData (TRAIN_DIR)

print("train: ", len(x_train))
print("valid: ", len(x_valid))

dog_files:  12500
cat_files:  12500
train:  280
valid:  120


In [ ]:
learning_rate = 0.0001
epochs = 20
batch_size = 10

### 2.3 dataset 생성

In [ ]:
# dataset 생성
train_dataset = data.Dataset_image([x_train, y_train], batch_size = batch_size)
valid_dataset = data.Dataset_image([x_valid, y_valid], batch_size = batch_size)

In [ ]:
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True)))
model = model.Vgg19(sess, "model")
model.build_net([150, 150, 3])

### 2.4 학습

In [ ]:
train.train (model, train_dataset, valid_dataset, learning_rate, epochs)

### 2.5 평가

In [ ]:
train.eval (model, train_dataset)

In [ ]:
train.eval (model, valid_dataset)